# Une prise en main rapide de Jupyter / Julia / JuMP

## Qu'est-ce qu'un Jupyter notebook ?

Un Jupyter notebook est un document qui contient 
+ du texte 
  - que l'on peut formatter à l'aide de Markdown
  - qui peut contenir des maths à l'aide de $\LaTeX$
+ du code
  - avec lequel on peut intéragir en ligne
  
Un notebook est une succession de cellule, chacune pouvant être soit du code, soit du texte.
Quelques astuces :
+ double-clicker pour rentrer dans une cellule
+ Ctrl-enter pour executer la cellule
+ Alt-enter pour executer la cellule et en ajouter une nouvelle

Vous pourrez travailler et rendre votre projet sous forme de Jupyter notebook.

## Qu'est-ce que JuliaBox ?

JuliaBox est un site internet vous offrant une session sur un ordinateur où Julia et Jupyter sont installés. De cette manière vous pouvez faire tourner votre code sans avoir besoin d'installer Julia sur votre ordinateur. Ainsi vous pourrez travailler sur votre projet toujours depuis votre navigateur préféré.

Attention, la session offert par JuliaBox dure 3h, au bout desquelles les calculs effectués sont perdus (mais pas le code). Donc, si d'un seul coup vous voyez des variables non-définies : pas d'affolement, il suffit de relancer le kernel.

## Qu'est-ce que Julia ?

Julia est un langage de programmation, comparable à Python. C'est un langage récent, développé pour le calcul scientifique. 

Quelques éléments intéressant :
+ langage open-source
+ langage compilé "Just-in-time"
+ langage disposant d'un terminal (comme python)
+ ...

Faisons nos premiers pas avec Julia.

In [2]:
1+1

2

In [3]:
a = [0 5 10 15]
a[1]

0

In [4]:
length(a)

4

In [5]:
for i = 1:5
    println("itération ",i)
end

itération 1
itération 2
itération 3
itération 4
itération 5


In [6]:
function factorielle(n)
    res = 1
    for i=1:n
        res = res * i
    end
    return res
end

factorielle (generic function with 1 method)

In [7]:
factorielle(5)

120



## Qu'est-ce que JuMP ?

JuMP est un package de Julia, qui est la raison pour laquelle ce projet sera réalisé en Julia.

Il s'agit d'un package de modélisation, qui permet d'écrire un problème d'optimisation de manière simple puis de demander à un Solver de le résoudre.

Nous allons maintenant faire no premiers pas avec JuMP.



### Mise en place

Avant toute chose il faut mettre à jour la liste des packets.

In [8]:
Pkg.update()

INFO: Updating METADATA...
INFO: Computing changes...
INFO: No packages to install, update or remove


Puis il faut installer JuMP

In [9]:
Pkg.add("JuMP")

INFO: Package JuMP is already installed


Installons également un solveur linéaire gratuit (cela prendra un peu de temps).

In [ ]:
Pkg.add("Clp")

Nous allons maintenant dire que nous souhaitons utiliser ces deux packages (comparable à "import" en python)

In [10]:
using JuMP, Clp


## Construction d'un premier problème linéaire

Nous souhaitons résoudre le problème linéaire suivant
$$ \begin{align*} 
\min_{x,y} \quad & 2x+3y \\
s.c. \quad & x+y \geq 1 \\
& x \geq 0, y\geq 0 \\
\end{align*}$$

Commençons par construire le problème

In [ ]:
SOLVER = ClpSolver()                    # On définit un solveur
m = Model(solver=SOLVER)                # On construit un problème d'optimisation

@variable(m,x>=0)                       # x est une variable réelle positive de m
@variable(m,y>=0)                       # y est une variable réelle positive de m

@objective(m,Min, 2*x+3*y)              # l'objectif de m est de Minimiser 2*x+3*y

@constraint(m,x+y >= 1 )                # m a pour contrainte x+y <=1

On peut vérifier que m est bien ce que l'on souhaite

In [ ]:
print(m)

On peut également résoudre m

In [ ]:
solve(m)

Et si on souhaite connaître la valeur optimale du problème ou des solutions optimales on peut les avoir de la manière suivante

In [ ]:
println(getobjectivevalue(m))
println("x = ",getvalue(x))
println("y = ",getvalue(y))

### Un second problème linéaire

Nous allons maintenant construire un problème linéaire plus complexe.
$$
\begin{align*}
\min_{x\in R^n} \quad & \sum_{i=1}^n c_i x_i \\
s.c. \quad & \sum_{i=1}^n x_i \geq n \\
&  -1 \leq x_i \leq 2 & \forall i
\end{align*}
$$

In [ ]:
m2 = Model(solver=SOLVER)
n = 10                                    # on choisit n = 10, mais vous pouvez le modifier
c = rand(n)                               # c est choisit ici de manière aléatoire                                              

@variable(m2, -1<= x[1:n] <= 2)           # x est une variable de m2 contenant n éléments x[1], x[2],...,x[n] tous compris entre -1 et 2

@objective(m2,Min, sum(c[i]*x[i] for i=1:n) )

@constraint(m2,sum(x[i] for i=1:n) >= n)
                        
print(m2)            

In [ ]:
solve(m2)

In [ ]:
getvalue(x)

Ajoutons maintenant une série de contraintes de la forme
$$ x_i + x_{i+1} \leq 1, \qquad \forall i \in 2, \dots, n-1$$

In [ ]:
for i = 1 : n-1
    @constraint(m2, x[i]+x[i+1] <= 2)
end

In [ ]:
solve(m2)
getvalue(x)

### Un problème non-linéaire

Nous allons terminer avec un problème non-linéaire simple.

$$
\begin{align*}
\min_{x \in R^{n.m}} \quad & \sum_{i,j} x_{i,j}^2 \\
s.c. \quad & \sum_{i=1}^n x_{i,j} = 1 & \forall j \in [m] 
\end{align*}
$$

In [11]:
Pkg.add("Ipopt")                   # Installons un solveur non-linéaire
using Ipopt

INFO: Package Ipopt is already installed


In [13]:
SOLVER_NL = IpoptSolver()

m3 = Model(solver = SOLVER_NL)

N,M = 5,7 

@variable(m3, x[i=1:N, j=1:M])

@objective(m3, Max, sum(x[i,j]^2 for i=1:N, j=1:M))

for j=1:M
    @constraint(m3, sum(x[i,j] for i =1:N)==1)
end

In [14]:
print(m3)

Max x[1,1]² + x[1,2]² + x[1,3]² + x[1,4]² + x[1,5]² + x[1,6]² + x[1,7]² + x[2,1]² + x[2,2]² + x[2,3]² + x[2,4]² + x[2,5]² + x[2,6]² + x[2,7]² + x[3,1]² + x[3,2]² + x[3,3]² + x[3,4]² + x[3,5]² + x[3,6]² + x[3,7]² + x[4,1]² + x[4,2]² + x[4,3]² + x[4,4]² + x[4,5]² + x[4,6]² + x[4,7]² + x[5,1]² + x[5,2]² + x[5,3]² + x[5,4]² + x[5,5]² + x[5,6]² + x[5,7]²
Subject to
 x[1,1] + x[2,1] + x[3,1] + x[4,1] + x[5,1] = 1
 x[1,2] + x[2,2] + x[3,2] + x[4,2] + x[5,2] = 1
 x[1,3] + x[2,3] + x[3,3] + x[4,3] + x[5,3] = 1
 x[1,4] + x[2,4] + x[3,4] + x[4,4] + x[5,4] = 1
 x[1,5] + x[2,5] + x[3,5] + x[4,5] + x[5,5] = 1
 x[1,6] + x[2,6] + x[3,6] + x[4,6] + x[5,6] = 1
 x[1,7] + x[2,7] + x[3,7] + x[4,7] + x[5,7] = 1
 x[i,j] ∀ i ∈ {1,2,3,4,5}, j ∈ {1,2,…,6,7}


In [15]:
solve(m3)
println(getobjectivevalue(m3))


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.1, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       35
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       35

Total number of variables............................:       35
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equa

In [16]:
getvalue(x)

5×7 Array{Float64,2}:
 0.2  0.2  0.2  0.2  0.2  0.2  0.2
 0.2  0.2  0.2  0.2  0.2  0.2  0.2
 0.2  0.2  0.2  0.2  0.2  0.2  0.2
 0.2  0.2  0.2  0.2  0.2  0.2  0.2
 0.2  0.2  0.2  0.2  0.2  0.2  0.2